# Task 4

Find the lowest eigenvalue of the following matrix:
  
               1    0    0   0
    A =        0    0   -1   0
               0   -1    0   0
               0    0    0   1
            

using VQE-like circuits, created from scratch. 



# Solution :

The steps followed to solve this task are listed below. This approach is further extended to a generalised form which can be used to calculate lowest eigenvalues for any 4x4 matrix with random values. The steps are as follows:
    
  1. Overview
  2. Tensor decomposition of Matrix A
  3. Calculating Ground state energy via Exacteigensolver
  4. Designing Ansatz
  5. Where we go from here

#  1. Overview

The Variational-Quantum-Eigensolver (VQE) is a quantum/classical hybrid algorithm that makes use of the variational principle to approximate the ground state of a given Quantum Hamiltonian. VQE allows us to find an upper bound of the lowest eigenvalue of a 
hamiltonian.
The variational principle ensures that,  given a certain quantum state $\left|\psi \right\rangle$, it satisfies the following inequivalence (we assume normalized kets):

$$
\left\langle \psi \right| H \left| \psi \right\rangle \leq E_0 \ ,
$$

where  E_0 \ = ground state energy           &        $\left|\psi \right\rangle$  corresponds to the ground state of the system

The variational principle provides an effective way to estimate the ground state of the system under investigation: by simply taking a set of kets $\left|\psi(\alpha , \, \beta , \, \dots) \right\rangle$, depending on some parameters $(\alpha , \, \beta , \, \dots)$, one can find the best approximation to the ground state by simply evaluating the expression $\left\langle \psi (\alpha , \, \beta , \, \dots) \right| H \left|\psi (\alpha , \, \beta , \, \dots) \right\rangle$ and looking for its minimum. To achieve this, we have to choose the ansatz $\left|\psi(\alpha , \, \beta , \, \dots) \right\rangle$.


# 2.  Tensor Decomposition of Matrix A

Since we want to measure a VQE-like circuit, we need to decompose the matrix into Pauli operators $\{\mathbb{1}, X, Y, Z\}$

$$
\mathbb{1} =
\begin{bmatrix}
1 & 0  \\
0 & 1
\end{bmatrix}\quad
X =
\begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}\quad
Y =
\begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix}\quad
Z =
\begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix}
$$

in order to implement it on a quantum computer. In particular, since we are dealing with a $4\times4$ matrix, two qubits are needed, thus the desired decomposition will be made of tensor products of two Pauli operators (e.g. $X_1 \otimes Y_2$, where the subscript denotes the system on which the operator acts).

Thus to get the pauli component of a $2^N$ x $2^N$ matrix $A$, we have:

$$
A = \sum_{ij} \frac{1}{4} h_{ij} \hspace{0.5em} \sigma_i \otimes \sigma_j
$$
And the components $h_{ij}$ are:

$$
h_{ij} = \frac{1}{4} \mathrm{Tr} \bigg[ (\sigma_i \otimes \sigma_j) \cdot A \bigg]
$$

Therefore we create a function get_components_from_matrix that does this decomposition for us.



# Imports

Here we mention all the imports we require :

In [1]:
import numpy as np
from random import random
from scipy.optimize import minimize
from sympy.solvers import solve 
from sympy import Symbol

from qiskit import *
from qiskit.tools.visualization import *
from qiskit.aqua.algorithms import ExactEigensolver
from qiskit.quantum_info.operators import Operator , Pauli
from qiskit.aqua.operators import WeightedPauliOperator


Let us convert the given matrix A as numpy array :

In [2]:
A = np.array([[1 , 0. , 0. , 0.], [0. , 0. , -1. , 0.] , [0. , -1. , 0. , 0.] , [0. , 0. , 0. , 1.]])


We now define a function which transforms our $4 \times 4$ matrix to tensor decomposition in terms of the Pauli operators.

In [3]:
def HO(matrix):          
    list_labels = []
    pauli_operator = []
    symbols = []
    equations = []
    list_dict = []
    unknown_matrix = 0
    
    #these are the building blocks of the Pauli objects we are going to have, like II
    pauli_mat = ('I','X','Y','Z') 
    for i in range(0,4):
        for j in range(0,4):
            #we create the list of the labels
            list_labels.append(pauli_mat[i] + pauli_mat[j])
            pauli_operator.append(Operator(Pauli(label=pauli_mat[i] + pauli_mat[j])))
            
    for i in range(0,16):
        # we create the list of symbols, which we need to solve the equations below
        symbols.append(Symbol(list_labels[i]))  
        
    for i in range(0,16):
        # we create the matrix containing the various pauli operators and the unknown
        unknown_matrix = unknown_matrix + (Symbol(list_labels[i])) * pauli_operator[i].data 
        
    for i in range(0, 4):
        for j in range(0,4):
            # we create the list of equations
            equations.append(unknown_matrix[i,j] - matrix[i,j]) 
    sol = solve(equations , symbols, manual = True)
    
    # we create the dictionary to give to the WeightedPauliOperator 
    for i in range(0,16):
        list_dict.append({"coeff": {"imag": 0.0, "real": sol[0][i]}, "label": list_labels[i]})
    pauli_dict = {
        'paulis': list_dict
    } 
    return WeightedPauliOperator.from_dict(pauli_dict)


and we are going to use this function to transform our matrix $A$ in the sum of Pauli operators

In [4]:
H = HO(A)


#  Calculation of Ground State Energy

For an $n$-qubit quantum state, measuring one qubit corresponds to projecting the quantum state onto one of two half-spaces determined by the unique eigenvalues of our measurement operator.

By convention, performing a computational basis measurement is equivalent to measuring in measuring Pauli Z, which gives us two eigenvectors $|0\rangle$ and $|1\rangle$, with corresponding eigenvalues $\pm 1$. Therefore, doing a computation measurement of a qubit and obtaining $0$ means the state of our qubit is in the $+1$ eigenstate of the $Z$ operator.

Now, we will calculate the ground state energy via the ExactEigensolver class

In [5]:
exact_eigensolver = ExactEigensolver(H)
result = exact_eigensolver.run()
reference_energy = result['energy']

C:\ProgramData\Anaconda3\lib\site-packages\qiskit\aqua\algorithms\eigen_solvers\numpy_eigen_solver.py:242: DeprecationWarning: Deprecated class ExactEigensolver, use NumPyEigensolver.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\qiskit\aqua\algorithms\eigen_solvers\eigen_solver_result.py:64: DeprecationWarning: energy deprecated, use eigenvalues property.
  warnings.warn('energy deprecated, use eigenvalues property.', DeprecationWarning)



# Designing Ansatz 

Ansatze are simply a parameterized quantum circuits (PQC), which plays an important role in the performance of HQC algorithms. Major challenge while designing an asatz is to choose an effective template circuit that well represents the solution space while maintaining a low circuit depth and number of parameters.

As already mentioned, a challenging aspect in the VQE algorithms is the choice of the ansatz: indeed, it is important to choose an ansatz which includes the ground state or very close to the true ground state of the system, in order to have a good estimate of the ground state energy.

To build the ground state via a quantum cirquit starting from the state $\left| 00 \right\rangle$, a Hadamard gate, and a subsequent CNOT gate are necessary, in order to make a superposition and to create the entanglement. Hence, we will work with the following ansatz:

$$
\left( R_z \left(\alpha\right) \, I \right)\left( R_x \left(\beta \right) \, I \right) CX \left( HI \right) \, \left| 00 \right\rangle \ ,
$$

where the last two gates determine the relative angle betweent the two qubits; out of which RX is variational here. Given this ansatz, we now define a function which, given two parameters, prepares the quantum state associated to the ansatz, together with the two classical bits for the measurements.



In [6]:
def real_ansatz(param):
    qc = QuantumCircuit(2,2)
    # Add a H gate on qubit 0, putting this qubit in a superposition.
    qc.h(0)
    # Add a CX (CNOT) gate on control qubit 0 and target qubit 1, putting
    # the qubits in a Bell state and creating then entanglement.
    qc.cx(0, 1)
    # and now we create the part of the Ansatz controlled by the parameters:
    #we do an Rx rotation and an Rz rotation (in this order!)
    qc.rx(param[0],0)
    qc.rz(param[1],0)
    return qc


Now, we will convert our hamiltonian i.e H in dictionary form by using Qiskit built-in method .to_dict()

In [7]:
H_dict = H.to_dict()
H_dict

{'paulis': [{'label': 'II', 'coeff': {'real': 0.5, 'imag': 0.0}},
  {'label': 'XX', 'coeff': {'real': -0.5, 'imag': 0.0}},
  {'label': 'YY', 'coeff': {'real': -0.5, 'imag': 0.0}},
  {'label': 'ZZ', 'coeff': {'real': 0.5, 'imag': 0.0}}]}



& now we will prepare a new function to get a list of dictionaries from H i.e. U_dict

In [8]:
def pauli_to_dict(pauli_operator):
    
    """
    This function eats our hamiltonian, constructed previously, and converts it to a list of dictionaries:
    [{'label': 'II', 'coeff': 0.5},
 {'label': 'XX', 'coeff': -0.5},
 {'label': 'YY', 'coeff': -0.5},
 {'label': 'ZZ', 'coeff': 0.5}].
    """
    
    dict_long = pauli_operator.to_dict()
    paulis = dict_long['paulis']
    paulis_dict = []

    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict.append({'label' : label , 'coeff' : coeff})

    return paulis_dict
U_dict = pauli_to_dict(H)


When working with dictionaries, it is good to have a function which, given a label, returns the value associated to that label or returns $0$ if that label does not exist in the dictionary

In [9]:

def get_or_else_zero(d: dict, key: str):
    
    """
    Utility for working with dictionaries. If key is missing
    than return 0 otherwise the corresponding value.
    :param dict: the dictionary.
    :param key: key (string) in interest.
    :return: 0 or value of corresponding key.
    """
    
    value = 0
    if key in d:
        value = d[key]
    return value


Now, we define a function that, given a circuit, a qubit, and a measurement to do, appends the gate associated with the measurements we are going to do on the qubit and connect the qubit to the corresponding classical bit

In [10]:
def measurement_circuit(circuit, measure : str, qubit):
    
    """
    Create the preparation for a measurement: it adds the gates 
    necessary to convert the circuit to the appropriate 
    computational basis and connect the qubits with the corresponding classical bits
    """

    if measure == 'Z':
        circuit.measure(qubit, qubit)
    elif measure == 'X':
        circuit.u2(qubit, np.pi, qubit)
        circuit.measure(qubit, qubit)
    elif measure == 'Y':
        circuit.u2(qubit, np.pi/2, qubit)
        circuit.measure(qubit, qubit)
    
    return circuit


Let us now define the function associated with the quantum part of the algorithm. This function prepares the quantum circuit and performs the quantum experiment associated with a certain measurement. In the case the measurement involves an identity operator on one of the two qubits, we do the measurement on the other qubit only

In [11]:
def quantum_mod(parameters, measure):
    
    # we check the presence of identity operators    
    if measure == 'II':
        return 1 # if both operators are identity there is nothing to measure
    elif measure[0] == 'I':
        check_identity = 2
    elif measure[1] == 'I':
        check_identity = 1
    else:
        check_identity = 0
    
    # quantum state preparation
    circuit = real_ansatz(parameters)
    
    for i in range(0,2):
        circuit = measurement_circuit(circuit, measure[i] , i)
    
    shots = 2000
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    if check_identity == 0:
        expectation_value = (get_or_else_zero(counts, '00') + get_or_else_zero(counts, '11')  - get_or_else_zero(counts,'01') -  get_or_else_zero(counts,'10')) / shots
    elif check_identity == 2:
        expectation_value = (get_or_else_zero(counts, '00') - get_or_else_zero(counts,'10')) / shots
    elif check_identity == 1:
        expectation_value = (get_or_else_zero(counts, '00') - get_or_else_zero(counts,'01')) / shots
    
    return expectation_value



Now, we will define the main function, which sums the results of the measurements perfomed on the various terms of the hamiltonian.

In [12]:
def vqe(parameters):
    
    classical_adder = 0
        
    for i in range(0, len(U_dict)):
        classical_adder = classical_adder + U_dict[i]['coeff'] * quantum_mod(parameters, U_dict[i]['label'])
    
    return classical_adder


Finally, we are ready to compute the ground state with the VQE algorithm. We use two methods: in the first we simply scan all the values of the parameters for our ansatz, in the second we use the minimize() method of scipy.

In [13]:
n_spacing_alpha = 25
n_spacing_beta = 25
parameters_array_alpha = np.linspace(0, 2 * np.pi, n_spacing_alpha)
parameters_array_beta = np.linspace(0, np.pi, n_spacing_beta)
sol_list = []

for i in range(0 , n_spacing_alpha):
    for j in range(0 , n_spacing_beta):
        sol_list.append(vqe([parameters_array_alpha[i] , parameters_array_alpha[j]]))

print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy using the VQE algorithm is: {}'.format((sorted(sol_list))[0]))

The exact ground state energy is: -0.9999999999999999
The estimated ground state energy using the VQE algorithm is: -1.0


In [14]:
parameters_array = np.array([np.pi, np.pi]) # starting point for the parameters
tol = 1e-5 # tolerance for the required precision.

vqe_result = minimize(vqe, parameters_array, method = 'Powell',tol=tol)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm and by minimization method is: {}'.format(vqe_result.fun))

The exact ground state energy is: -0.9999999999999999
The estimated ground state energy from VQE algorithm and by minimization method is: -1.0



as we can see, the quality of the approximation is extremely good. This is not surprising, since the ground state falls in the ansatz we used.


# Where we go from here :


We can generalize a bit our algorithm and consider more general matrices: we keep a matrix with the same structure of non-vanishing coefficients, but we replace the coefficients with random numbers (but we impose the restriction that the matrix is symmetric). We can also add few terms on the main diagonal.

In [15]:
x = random()
y = random()

A = np.array([[x , 0. , 0. , 0.], [0. , x , y , 0.] , [0 , y , x, 0.] , [0. , 0. , 0. , y]])
H = HO(A)

exact_eigensolver = ExactEigensolver(H)
result = exact_eigensolver.run()
reference_energy = result['energy']

U_dict = pauli_to_dict(H)
U_dict

C:\ProgramData\Anaconda3\lib\site-packages\qiskit\aqua\algorithms\eigen_solvers\numpy_eigen_solver.py:242: DeprecationWarning: Deprecated class ExactEigensolver, use NumPyEigensolver.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\qiskit\aqua\algorithms\eigen_solvers\eigen_solver_result.py:64: DeprecationWarning: energy deprecated, use eigenvalues property.
  warnings.warn('energy deprecated, use eigenvalues property.', DeprecationWarning)


[{'label': 'II', 'coeff': 0.1583376005168618},
 {'label': 'IZ', 'coeff': -0.0618210635387664},
 {'label': 'XX', 'coeff': 0.1719003955665805},
 {'label': 'YY', 'coeff': 0.1719003955665805},
 {'label': 'ZI', 'coeff': -0.0618210635387664},
 {'label': 'ZZ', 'coeff': 0.0618210635387664}]

In [16]:
n_spacing_alpha = 25
n_spacing_beta = 25
parameters_array_alpha = np.linspace(0, 2 * np.pi, n_spacing_alpha)
parameters_array_beta = np.linspace(0, np.pi, n_spacing_beta)
sol_list = []

for i in range(0 , n_spacing_alpha):
    for j in range(0 , n_spacing_beta):
        sol_list.append(vqe([parameters_array_alpha[i] , parameters_array_alpha[j]]))

print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy using the VQE algorithm is: {}'.format((sorted(sol_list))[0]))

The exact ground state energy is: -0.24728425415506547
The estimated ground state energy using the VQE algorithm is: -0.2422767480084255


In [17]:
x = random()
y = random()
z = random()

A = np.array([[0. , x , 0. , 0.], [0. , x , y , 0.] , [0 , y , x, z] , [0. , 0. , 0. , z]])
H = HO(A)

exact_eigensolver = ExactEigensolver(H)
result = exact_eigensolver.run()
reference_energy = result['energy']

U_dict = pauli_to_dict(H)
U_dict


C:\ProgramData\Anaconda3\lib\site-packages\qiskit\aqua\algorithms\eigen_solvers\numpy_eigen_solver.py:242: DeprecationWarning: Deprecated class ExactEigensolver, use NumPyEigensolver.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\qiskit\aqua\algorithms\eigen_solvers\eigen_solver_result.py:64: DeprecationWarning: energy deprecated, use eigenvalues property.
  warnings.warn('energy deprecated, use eigenvalues property.', DeprecationWarning)


[{'label': 'II', 'coeff': 0.27688655717123173},
 {'label': 'IX', 'coeff': 0.15127459341902824},
 {'label': 'IY', 'coeff': 0.0},
 {'label': 'IZ', 'coeff': -0.02566262966682475},
 {'label': 'XX', 'coeff': 0.3598785657742115},
 {'label': 'YY', 'coeff': 0.3598785657742115},
 {'label': 'ZI', 'coeff': -0.02566262966682475},
 {'label': 'ZX', 'coeff': 0.09994933408537875},
 {'label': 'ZY', 'coeff': 0.0},
 {'label': 'ZZ', 'coeff': -0.22556129783758225}]

In [18]:
n_spacing_alpha = 25
n_spacing_beta = 25
parameters_array_alpha = np.linspace(0, 2 * np.pi, n_spacing_alpha)
parameters_array_beta = np.linspace(0, np.pi, n_spacing_beta)
sol_list = []

for i in range(0 , n_spacing_alpha):
    for j in range(0 , n_spacing_beta):
        sol_list.append(vqe([parameters_array_alpha[i] , parameters_array_alpha[j]]))

print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy using the VQE algorithm is: {}'.format((sorted(sol_list))[0]))

The exact ground state energy is: -0.3975270523209651
The estimated ground state energy using the VQE algorithm is: -0.2166285394930849



#### That's all !! 